<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Treinamento_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configurações Iniciais 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
import string, re
import itertools
import nltk
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping
py.init_notebook_mode(connected=True)
%matplotlib inline
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

## Lendo a Base

In [3]:
# Base1 Com StopWords
PATH_BASE1 = "/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv"

# Base2 Sem StopWords
PATH_BASE2 = "/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv"

# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA = "/content/drive/MyDrive/TCC/dados/processadas/Base1JuntaHateENotHateDaLeila.csv"

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA = "/content/drive/MyDrive/TCC/dados/processadas/Base2_maior4_menor25_limpissima.csv"

In [4]:
#trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA, index_col=0)
trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA)

In [5]:
# Agrupa dados por label
trainDF.groupby('label').count()

,text
label,
hate,2443
not_hate,19282


In [6]:
categorias_desejadas = ['hate', 'not_hate'] # !!!
df_filtrado = trainDF[trainDF.label.isin(categorias_desejadas)]
df_filtrado.shape

(21725, 2)

In [7]:
df_filtrado

,label,text
0,not_hate,geral foi no jogo do foz
1,not_hate,pois é não é gente é isso ai pessoal
2,not_hate,ontem eu estava tão mal que escrevi tudo o que...
3,not_hate,eu devo ter um coração de pedra não sou de me ...
4,not_hate,camimq vivobr pois é estou tentando entender
...,...,...
21720,hate,eduguim depois falam que nordestino não sabe ...
21721,hate,e dizem por aí que o nordestino não sabe votar
21722,hate,ain que nordestino não sabe votar e blá blá blá
21723,hate,imagine essa turma da extrema direita soltando...


In [8]:
def balanceia_dados(df, verbose=0):
    # Esta funcao recebe um dataframe original e retorna um dataset
    # contendo um numero igual de noticias das categorias
    # por simplicidade, vou supor que são somente duas categorias.
    global categorias_desejadas
    df_balanceado = df.copy()
    
    # cria um dataframe para cada classe. Assim podemos remover o excesso
    df_F = df_balanceado.loc[df_balanceado.label == categorias_desejadas[0]]
    df_T = df_balanceado.loc[df_balanceado.label == categorias_desejadas[1]]

    num_amostras_F, num_amostras_T = df_F.shape[0], df_T.shape[0]
    num_amostras_a_remover = abs(num_amostras_F - num_amostras_T)
    
    # remove amostras da classe majoritaria
    if num_amostras_F > num_amostras_T:
        #print(df_F[:num_amostras_a_remover)
        df_F = df_F[num_amostras_a_remover:]
        
    elif num_amostras_T > num_amostras_F:
        #
        df_T = df_T[num_amostras_a_remover:]

    return pd.concat([df_F, df_T])

In [9]:
# Aplica função de balancear os dados
dataBC = balanceia_dados(df_filtrado)

# DATA BALANCEADO
trainDF = dataBC
trainDF.reset_index(drop=True, inplace=True) # reseta index
trainDF.shape

(4886, 2)

In [10]:
# convert output  hate =1 not_hate =0
trainDF.label = [1 if each == "hate" else 0 for each in trainDF.label] 

In [11]:
trainDF.groupby('label').count()

,text
label,
0,2443
1,2443


In [12]:
# Pega a média de caracteres dos tweets de toda a base
average_len = int(trainDF['text'].apply(lambda x: len(str(x))).mean())
average_len

62

In [13]:
palavras_unicas = list ( trainDF['text'].str.split(' ', expand=True).stack().unique() ) 
quantidade_palavras_unicas = len( palavras_unicas )
quantidade_palavras_unicas

9144

In [51]:
oi = set(palavras_unicas)
len(oi)

9144

#TF-IDF

In [63]:
# word level tf-idf
# td-idf gera matriz com 9140 colunas (4 palavras estão sendo perdidas ao usar o regex \w{1,})
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features= quantidade_palavras_unicas)
X = tfidf_vect.fit_transform(trainDF['text'])

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, trainDF['label'], test_size=0.3, random_state=2)

# CNN model

In [67]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label, epochs=10) #adicionamos epoca
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [71]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((24, ))

    # Add the word embedding Layer
    #embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=True)(input_layer)
    #embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(input_layer)
    #conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(64, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    #sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='sgd', loss='mean_squared_error', metrics = ['acc', 'mae'])
  
    return model

In [72]:
classifier = create_cnn()
#accuracyCNN = train_model(classifier,train_seq_x, train_y, valid_seq_x, is_neural_net=True)
accuracyCNN = train_model(classifier,train_x, train_y, valid_x, is_neural_net=True)
print ("CNN, Word Embeddings - accuracy",  accuracyCNN)

ValueError: ignored